In [18]:
# key variables
from_num_iters=400000
model_name = 'Default1Hot'
one_hot=True

In [19]:
# CONVERSION PERFORMS THE ACTUAL VOICE CONVERSION THAT HAPPENS AFTER A MODEL IS TRAINED,
# SO WE'VE BEEN PROVIDED WITH A PRETRAINED AUTOVC MODEL TO DEMONSTRATE THIS

import os
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator
import random

def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

# open the data that was used to train the model - consists of id, embedding, uttrs file name
all_training_data = pickle.load(open('./model_saves/' +model_name +'/training_meta_data.pkl', 'rb'))
all_meta_data = pickle.load(open('all_meta_data.pkl', "rb"))
one_hot_array = np.eye(len(all_training_data))[np.arange(len(all_training_data))]

In [20]:
# meta_spkr_list = []
# metadata = pickle.load(open('metadata.pkl', "rb"))
# for entry in metadata:
#     meta_spkr_list.append(entry[0])
# for entry in all_training_data:
#     if entry[0] in meta_spkr_list:
#         print(f'{entry[0]} from training is in metadata')

In [21]:
# select random males and females from the training set
selected_speaker_for_conversion = []
num_spkrs_per_gender = 2
spkrs_found = 0
uttr_idx = '001'

male_id_list = []
female_id_list = []
f = open("/import/c4dm-datasets/VCTK-Corpus-0.92/speaker-info.txt", "r")
# print(f.read())
for i, x in enumerate(f):
    if i==0:
        continue
    female_id_list.append(x[0:4]) if x[10] =='F' else male_id_list.append(x[0:4])
print('MALES', male_id_list)
print('FEMALES', female_id_list)
gender_lists = [male_id_list, female_id_list]


# collect list of training speaker IDs
all_training_id = []
for spkr_data in all_training_data:
    all_training_id.append(spkr_data[0])
all_training_id

# randomly select males and females from dataset
random_spkrs = []
for gender_list in gender_lists:
    while spkrs_found < num_spkrs_per_gender:
        random_spkr = random.choice(gender_list)
        melspec_path ='./spmel/' +random_spkr +'/' +random_spkr +'_' +uttr_idx +'_mic1.npy'
        # check to see does path with random speaker and selected utterance exist
        if os.path.exists(melspec_path):        
            if one_hot==True:
                # check that selected speaker is in training set
                if random_spkr in all_training_id:
                    random_spkrs.append(random_spkr)
                    spkrs_found += 1
            else:
                random_spkrs.append(random_spkr)
                spkrs_found += 1
    spkrs_found = 0
print('selected random_spkrs', random_spkrs)

# save appropriate speaker metadata to a list

for selected_speaker in random_spkrs:
    print('selected_speaker', selected_speaker)
    if one_hot==True:
        for i, spkr_data in enumerate(all_training_data):
            if selected_speaker == spkr_data[0]:
#                 print(selected_speaker,'is in all_training_data')
                spkr_id = spkr_data[0]
                if one_hot==True:
                    spkr_emb = one_hot_array[i]
                else:
                    spkr_emb = spkr_data[1]
                melspec_path ='./spmel/' +spkr_id +'/' +spkr_id +'_' +uttr_idx +'_mic1.npy'
                melspec = np.load(melspec_path)
                selected_speaker_for_conversion.append((spkr_id, spkr_emb, melspec))    
            else:
                tester = 0
#                 print(selected_speaker,'not in all_training_data')
    else:
        # need to get embeddings for
        for spkr_data in all_meta_data:
            if selected_speaker == spkr_data[0]:
#                 print(selected_speaker,'is in all_meta_data')
                spkr_id = spkr_data[0]
                if one_hot==True:
                    spkr_emb = one_hot_array[i]
                else:
                    spkr_emb = spkr_data[1]
                melspec_path ='./spmel/' +spkr_id +'/' +spkr_id +'_' +uttr_idx +'_mic1.npy'
                melspec = np.load(melspec_path)
                selected_speaker_for_conversion.append((spkr_id, spkr_emb, melspec))
            else:
                tester = 0
#                 print(selected_speaker,'not in all_metadata_data')

MALES ['p226', 'p227', 'p232', 'p237', 'p241', 'p243', 'p245', 'p246', 'p247', 'p251', 'p252', 'p254', 'p255', 'p256', 'p258', 'p259', 'p260', 'p263', 'p270', 'p271', 'p272', 'p273', 'p274', 'p275', 'p278', 'p279', 'p281', 'p284', 'p285', 'p286', 'p287', 'p292', 'p298', 'p302', 'p304', 'p311', 'p315', 'p316', 'p326', 'p334', 'p345', 'p347', 'p360', 'p363', 'p364', 'p374', 'p376']
FEMALES ['p225', 'p228', 'p229', 'p230', 'p231', 'p233', 'p234', 'p236', 'p238', 'p239', 'p240', 'p244', 'p248', 'p249', 'p250', 'p253', 'p257', 'p261', 'p262', 'p264', 'p265', 'p266', 'p267', 'p268', 'p269', 'p276', 'p277', 'p280', 'p282', 'p283', 'p288', 'p293', 'p294', 'p295', 'p297', 'p299', 'p300', 'p301', 'p303', 'p305', 'p306', 'p307', 'p308', 'p310', 'p312', 'p313', 'p314', 'p317', 'p318', 'p323', 'p329', 'p330', 'p333', 'p335', 'p336', 'p339', 'p340', 'p341', 'p343', 'p351', 'p361', 'p362', 's5  ']
selected random_spkrs ['p364', 'p376', 'p293', 'p233']
selected_speaker p364
selected_speaker p376
selec

In [22]:
# if using one-hot, we can't use the metadata
device = 'cuda:0'
# G = Generator(32,256,512,32).eval().to(device)

#NEW BIT
if one_hot==False:
    G = Generator(32,256,512,32).eval().to(device)
else:
    G = Generator(32,20,512,32).eval().to(device)

g_checkpoint = torch.load('./model_saves/' +model_name +'/ckpts/ckpt_' +str(from_num_iters) +'.pth.tar')
G.load_state_dict(g_checkpoint['model_state_dict'])

################### Paper - Section 4.2. The Content Encoder #########################################################

# spect_vc collects spectrogram information to be used later for spec2wav conversion via wavnet model
spect_vc = []

x_org_list = []

# each sbmt_i has a speaker ID, a speaker embedding, and a spectrogram
for sbmt_i in selected_speaker_for_conversion:
    
    # x origin - 80Mel spectrogram
    x_org = sbmt_i[2]
    x_org_list.append( ('{}'.format(sbmt_i[0]), x_org) )
    x_org, len_pad = pad_seq(x_org)
    # utterance origin is just padded spec in tensor form
    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device)
    # speaker embedding 
    if one_hot==True:
        emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device).float()
    else:
        emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device)
        
    # for each entry in metadata, use it as the embedding target and use this in G
    for sbmt_j in selected_speaker_for_conversion:
        # embedding target represents S2 in the paper - the embedding provided by a pretrained Speaker Encoder
        # I guess this converts each utterance from the data so that it matches the utterance of every speaker
        # So if there were 4 utterances with different speakers, then this code will generate 4*4=16 conversions
        if one_hot==True:
            emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device).float()
        else:
            emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        # as this is conversion not training, no backprop/gradientCommunication needed here
        with torch.no_grad():
            # x_identic_psnt = target utterance, produced by the Generator
            # Generator is fed an utterance and speaker 
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            # utterance target is the converted speech
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else:
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        # utterance is saved in a list, along with source and target ID
        spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )
        
        
with open('results.pkl', 'wb') as handle:
    pickle.dump(spect_vc, handle)

with open('x_org.pkl', 'wb') as howdy:
    pickle.dump(x_org_list, howdy)

print('done')

done


In [23]:
#takes approximately 1.5 hours
import torch
import librosa
import soundfile as sf
import pickle
from synthesis import build_model
from synthesis import wavegen
import time

start_time = time.time()

# supposedly trained on the VCTK model
device = torch.device("cuda")
model = build_model().to(device)
checkpoint = torch.load("checkpoint_step001000000_ema.pth")
model.load_state_dict(checkpoint["state_dict"])

counter = 0
spect_vc = pickle.load(open('results.pkl', 'rb'))

for spect in spect_vc:
#     if counter == 16:
#         break
#     counter +=1
#     if counter == 0:
#         continue
#     name = str(counter) + spect[0]
    name = spect[0]
    # this splits the numpy spectrograms in half to as to save processing time
    c = spect[1]
    print(name)
    waveform = wavegen(model, c=c)
#     librosa.output.write_wav(name+'.wav', waveform, sr=16000)
    sf.write('./model_saves/' +model_name +'/generated_wavs/' +name +'_' +str(from_num_iters) +'kIters.wav', waveform, samplerate=16000)

time_taken = time.time() - start_time
print(time_taken)

  0%|          | 7/37888 [00:00<09:01, 69.98it/s]

p364xp364


 18%|█▊        | 6828/37888 [01:38<07:29, 69.07it/s]


KeyboardInterrupt: 

In [1]:
#     import os
#     from IPython.display import Audio
#     wavs=[]

#     for file in os.listdir("./"):
#         if file.endswith(".wav"):
#             wavs.append(file)

#     for wav in wavs:
#         print(wav)

In [27]:
import pickle
spect_vc = pickle.load(open('results.pkl', 'rb'))
spect_vc[0]

('.xp225',
 array([[0.28224188, 0.20554663, 0.15181297, ..., 0.00463975, 0.00347378,
         0.01703978],
        [0.33337137, 0.23668179, 0.16679963, ..., 0.03454047, 0.02398722,
         0.02410528],
        [0.37502947, 0.2653952 , 0.19988462, ..., 0.05794184, 0.04958811,
         0.04577218],
        ...,
        [0.6726218 , 0.7531079 , 0.77818435, ..., 0.3141644 , 0.30782175,
         0.30555806],
        [0.6982055 , 0.75408345, 0.7720312 , ..., 0.34722567, 0.34975088,
         0.34997335],
        [0.64091647, 0.6756678 , 0.7091046 , ..., 0.34009993, 0.35053512,
         0.34477887]], dtype=float32))

In [ ]:
sound_file = './0xp225.wav'

Audio(data=sound_file, autoplay=True)

In [12]:
x_orgs = pickle.load(open('x_org.pkl', 'rb'))

for x_org in x_orgs:
    name = x_org[0]
    c = x_org[1]
    print(name)
    waveform = wavegen(model, c=c)   
#     librosa.output.write_wav(name+'.wav', waveform, sr=16000)
    sf.write(name+'.wav', waveform, samplerate=16000)

p225


  0%|          | 6/22784 [00:00<06:55, 54.88it/s]

p228


  0%|          | 8/19200 [00:00<04:03, 78.98it/s]

p256


  0%|          | 6/27904 [00:00<08:22, 55.48it/s]

p270


100%|██████████| 27904/27904 [06:03<00:00, 76.83it/s]


[33, 91, 83, 82, 7, 63, 16, 11, 53, 97, 79, 95, 2, 72, 26, 18, 69, 31, 60, 50]